In [9]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/n

True

In [24]:

#Trainer 
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
#WordNetLemmatizer reduce the word to stem so that it wont loose performance

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())
#reading the json file as text


#create 3 empty lists
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']


''' 
How it works?:
Look inside the intents in the json file and through the patterns. Then,
append the words into the word list.
'''
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize = splits up sentences into words
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        #(word_list) is a tuple. Tuple: stores multiple items into a single variable
        #(word_list) belongs to the category intent['tag']
        documents.append((word_list, intent['tag']))

        #check if the class is in the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

    '''Before building a Neural Network'''
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential() #sequential model
#              128 = Neurons  input_shape dependant on  the size of the training data of train_x
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model')


# hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size=5, verbose=1)
# model.save('chatbotmodel.h5', hist) #save the training data into a h5 file... I think?
print("Done")


Epoch 1/200


<ipython-input-24-6f0143f9e36a>:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


23/23 [==============================] - 1s 2ms/step - loss: 3.4180 - accuracy: 0.0442 
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 3.3498 - accuracy: 0.0973
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 3.2733 - accuracy: 0.0885
Epoch 4/200
23/23 [==============================] - 0s 3ms/step - loss: 3.2550 - accuracy: 0.0885
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1506 - accuracy: 0.1504
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1344 - accuracy: 0.1150
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 3.0477 - accuracy: 0.1858
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9295 - accuracy: 0.1947
Epoch 9/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7931 - accuracy: 0.2743
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7399 - accuracy: 0.2743
Epoch 11/200
23/23 [

In [34]:
#Chat Bot 
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.model')  #The output will be numerical data

#Clean up the sentences
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return  sentence_words

#Converts the sentences into a bag of words
def bag_of_words(sentence):
    sentence_words = clean_up_sentence()
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence) #bow: Bag Of Words, feed the data into the neural network
    res = model.predict(np.array([bow]))[0] #res: result. [0] as index 0
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverser=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0], 'probability': str(r[1])]})
    return return_list

 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

